# **Procedimiento para realizar consultas (requests) a la base de datos de EDAMAM mediante las API's :**

## **1. Recipe Search**
## **2. Nutrition Analysis**
## **3. Food Database**         

## *Por: Samuel Cortez / Maestría en Ciencia de Datos*

In [3]:
# Importar librerías
import pandas as pd
import requests
import json         
import logging

In [15]:
# Se define clase Edamam_mcd
class Edamam_mcd:
    "API que regresa datos de tipo json"
    def __init__(self, 
                id_nutrition, 
                key_nutrition, 
                id_recipes,
                key_recipes,
                id_food,
                key_food):
        
        # Asignación de claves / ID de las API's
        self.id_nut = id_nutrition
        self.key_nut = key_nutrition
        self.id_rec = id_recipes
        self.key_rec = key_recipes
        self.id_food = id_food
        self.key_food = key_food
    
    def Nut_Analysis(self, query):
        self.Nut_ingredients = query
        # Se define url (Access Point) para realizar consulta a la API Nutrition Analysis
        url_req_nutr = 'https://api.edamam.com/api/nutrition-data'
        # Se definen los parametros a pasar por la función get para realizar consulta
        params_nutr= {
            'app_id':self.id_nut,
            'app_key':self.key_nut,
            'ingr':self.Nut_ingredients
            }
        # Envío de solicitud para realizar consulta de análisis nutricional
        Response = requests.get(url_req_nutr, params = params_nutr)

        # Imprimir el código del estatus
        print("Status Code (Nutrition Analysis API):", Response.status_code)
        
        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Nutrition Analysis API)'.format(key=self.key_nut))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado
        self.r_Nut = Response.json()
        return self.r_Nut

    def Search_recipe(self, query):
        self.query_recipe = query
        # Se construye url con los parámetros para realizar consulta para busqueda de recetas de un platillo
        url_recipe = 'https://api.edamam.com/search?q={query}&app_id={id}&app_key={key}'.format(id=self.id_rec, key=self.key_rec,query=self.query_recipe)    
        Response = requests.get(url_recipe)

        # Imprimir el código de estatus
        print("Status Code (Recipe Search API) :", Response.status_code)
        
        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Recipe Search API)'.format(key=self.key_rec))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado 
        self.r_Recipe = Response.json()
        return self.r_Recipe

    def Search_food(self, query):
        self.query_food = query
        # Se define url (Access Point) para realizar consulta de análisis de comida
        url_food = 'https://api.edamam.com/api/food-database/parser?'
        # Se definen los parametros a pasar por la función get para realizar consulta
        params_food = {
            'app_id':self.id_food,
            'app_key':self.key_food,
            'ingr':self.query_food
            }
        # Se realiza una consulta dando la url
        Response = requests.get(url_food, params= params_food)
        
        # Imprimir el código de estatus
        print("Status Code (Food Database API):", Response.status_code)

        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Food Database API)'.format(key=self.key_rec))

        # Valor de retorno de tipo diccionario, Response tiene contenido JSON serializado
        self.r_Food = Response.json()
        return self.r_Food
    
    # Conjunto de funciones para generar dataframes con los datos principales de las consultas

    def Nutrient_Guide(self):
        # Manejo de excepciones
        try:
            # Se crean DataFrames de las variables totalNutrients, totalDaily y totalNutrientsKCal 
            self.df_Nutrition= pd.DataFrame(self.r_Nut.get('totalNutrients')).T.rename_axis(columns= str(self.Nut_ingredients))
            self.df_totalDaily = pd.DataFrame(self.r_Nut.get('totalDaily')).T.rename_axis(str(self.Nut_ingredients))
            self.df_total_Nut =  pd.DataFrame(Response_Nut.get('totalNutrientsKCal')).T.rename_axis(str(self.Nut_ingredients))
            # Se obtiene las calorías del alimento
            self.Nutrient_Cal = self.r_Nut.get('calories')
            # Se obtiene el peso total del alimento
            self.totalWeight = self.r_Nut.get('totalWeight')
            
        except Exception as e:
            print(e)
        
    def ingredients_table(self):
        # Manejo de excepciones
        try:
            # Comprensión de listas: se crean nuevas listas con las etiquetas de la receta y con los ingredientes
            list_label = [element_hits.get('recipe').get('label') for element_hits in Response_Recipe.get('hits')]
            list_ingredientes = [element_ingre.get('recipe').get('ingredients') for element_ingre in Response_Recipe.get('hits')]
            # Se construye DataFrame 
            df_ = pd.DataFrame(list_ingredientes, index=list_label).stack().apply(pd.Series).drop(columns=['foodId'])
            self.df_Recipe = df_.rename_axis(["Types", "Items"], axis = "rows").rename(columns = {"foodCategory": "food category", "text":"ingredient name"})

        except Exception as e:
            print(e)

    def food_table(self):
        # Se generar listas vacias
        list_foods = []
        list_nutrients = []
        rename_nutrients = ['ENERGY (kcal)', 'PROTEIN (g)', 'FAT (g)', 'CARBS (g)', 'FIBER (g)']
        # Manejo de excepciones
        try:
            # Comprensión de listas: se crean nuevas listas con las etiquetas de los alimentos y con los nutrientes  
            list_foods = [element.get('food').get('label') for  element in self.r_Food['hints']]
            list_nutrients = [element.get('food').get('nutrients') for element in self.r_Food['hints']]
            # Se construye DataFrame
            self.df_food_table = pd.DataFrame(list_nutrients, index = list_foods).round(2).fillna('Unknown')
            self.df_food_table.columns = rename_nutrients
            
        except Exception as e:
            print(e) 
    
    def write_files(self):
        #  Exportar archivos .CSV/.xlsx
        path_Nutrition = '{Nut}_Nutritional_Analysis.xlsx'.format(Nut=self.Nut_ingredients)
        path_Recipe = '{Recipe}_Recipe.csv'.format(Recipe = self.query_recipe)
        path_food = '{food}_food.csv'.format(food = self.query_food)

        # Se exporta a excel el análisis nutricional 
        with pd.ExcelWriter(path_Nutrition) as writer:
            self.df_Nutrition.to_excel(writer, sheet_name= 'Nutritional_Analysis')
            self.df_totalDaily.to_excel(writer, sheet_name='Total_Daily')
            self.df_total_Nut.to_excel(writer, sheet_name='totalNutrientsKCal')
        # Se exporta en formato CSV los ingredientes de las recetas
        self.df_Recipe.to_csv(path_Recipe)
        # Se exporta en CSV el dataframe con el análisis de los alimentos
        self.df_food_table.to_csv(path_food)
        

In [5]:
# Claves / ID's de las aplicaciones asignadas
nutrition_appid= '5f1e7888'
nutrition_appkey= '5576413f5acb38f5259a58dad888d1b9'
recipes_appid = 'dcda2d5e'
recipes_appkey= '87ae2d34cdde47026fc341c8fbd7957f'
food_appid= '6b69c52c'
food_appkey= 'd5a448b534a0c358dfb47fa7a6548724'

# Se crea el objeto 
EDAMAM_consulta = Edamam_mcd(id_nutrition=nutrition_appid, 
                            key_nutrition= nutrition_appkey,
                            id_recipes=recipes_appid,
                            key_recipes=recipes_appkey,
                            id_food=food_appid,
                            key_food=food_appkey)

# Ejercución de los métodos de cada una de las API's para realizar consultas
Response_Nut = EDAMAM_consulta.Nut_Analysis(query= '2 hams')
Response_Food = EDAMAM_consulta.Search_food(query= 'coffe and pizza')
Response_Recipe = EDAMAM_consulta.Search_recipe(query= 'Twinkies')
# Ejecución de los métodos para obtener los dataframe con los resultados para cada una de las API's
EDAMAM_consulta.Nutrient_Guide()
EDAMAM_consulta.ingredients_table()
EDAMAM_consulta.food_table()
# Se exportan en archivos csv y xlsx los resultados obtenidos
EDAMAM_consulta.write_files()


Status Code (Nutrition Analysis API): 200
Status Code (Food Database API): 200
Status Code (Recipe Search API) : 200


In [6]:
# Consulta a la API: Nutrition Analysis obtenida en formato json
Response_Nut

{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_70bb57b7624646c4a7b643c2d9637fee',
 'calories': 29354,
 'totalWeight': 16308.0,
 'dietLabels': ['LOW_CARB'],
 'healthLabels': ['SUGAR_CONSCIOUS',
  'LOW_SUGAR',
  'PALEO',
  'DAIRY_FREE',
  'GLUTEN_FREE',
  'WHEAT_FREE',
  'EGG_FREE',
  'MILK_FREE',
  'PEANUT_FREE',
  'TREE_NUT_FREE',
  'SOY_FREE',
  'FISH_FREE',
  'SHELLFISH_FREE',
  'CRUSTACEAN_FREE',
  'CELERY_FREE',
  'MUSTARD_FREE',
  'SESAME_FREE',
  'LUPINE_FREE',
  'MOLLUSK_FREE',
  'ALCOHOL_FREE',
  'NO_OIL_ADDED',
  'NO_SUGAR_ADDED',
  'SULPHITE_FREE'],
 'cautions': ['SULFITES'],
 'totalNutrients': {'ENERC_KCAL': {'label': 'Energy',
   'quantity': 29354.4,
   'unit': 'kcal'},
  'FAT': {'label': 'Fat', 'quantity': 1679.7240000000002, 'unit': 'g'},
  'FASAT': {'label': 'Saturated', 'quantity': 558.0597600000001, 'unit': 'g'},
  'FAMS': {'label': 'Monounsaturated',
   'quantity': 798.6027600000001,
   'unit': 'g'},
  'FAPU': {'label': 'Polyunsaturated',
   'quantity': 2

In [7]:
# Consulta a la API: Food Database obtenida en formato json
Response_Food

{'text': 'coffe and pizza',
 'parsed': [{'food': {'foodId': 'food_at830s9amds32fb8w6ufmaopzk8n',
    'label': 'Pizza',
    'nutrients': {'ENERC_KCAL': 268.0,
     'PROCNT': 10.36,
     'FAT': 12.28,
     'CHOCDF': 29.02,
     'FIBTG': 2.2},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/c01/c0150280d71059c23c025f501f502dc0.jpg'}}],
 'hints': [{'food': {'foodId': 'food_at830s9amds32fb8w6ufmaopzk8n',
    'label': 'Pizza',
    'nutrients': {'ENERC_KCAL': 268.0,
     'PROCNT': 10.36,
     'FAT': 12.28,
     'CHOCDF': 29.02,
     'FIBTG': 2.2},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/c01/c0150280d71059c23c025f501f502dc0.jpg'},
   'measures': [{'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_unit',
     'label': 'Whole'},
    {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_pizza',
     'label': 'Pizza'},
    {'uri': 'http://www.edamam.com/o

In [8]:
# Consulta a la API: Recipe Search obtenida en formato json
Response_Recipe

{'q': 'Twinkies',
 'from': 0,
 'to': 10,
 'more': True,
 'count': 54,
 'hits': [{'recipe': {'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_38aa4916e63dd6cfdf2c4340f88a4b4b',
    'label': 'Twinkie Tiramisu recipes',
    'image': 'https://www.edamam.com/web-img/14d/14dd6585a1d51c813af62d77b00a18e9',
    'source': 'My Recipes',
    'url': 'http://www.myrecipes.com/recipe/twinkie-tiramisu',
    'shareAs': 'http://www.edamam.com/recipe/twinkie-tiramisu-recipes-38aa4916e63dd6cfdf2c4340f88a4b4b/twinkies',
    'yield': 8.0,
    'dietLabels': ['Low-Sodium'],
    'healthLabels': ['Low Potassium',
     'Kidney-Friendly',
     'Vegetarian',
     'Pescatarian',
     'Gluten-Free',
     'Wheat-Free',
     'Egg-Free',
     'Peanut-Free',
     'Tree-Nut-Free',
     'Soy-Free',
     'Fish-Free',
     'Shellfish-Free',
     'Pork-Free',
     'Red-Meat-Free',
     'Crustacean-Free',
     'Celery-Free',
     'Mustard-Free',
     'Sesame-Free',
     'Lupine-Free',
     'Mollusk-Free',
     'Alc

## **Despliegue de los DataFrames de la API Nutrition Analysis**

In [9]:
# Desplegar DataFrame de Nutrientes totales en kCal
EDAMAM_consulta.df_total_Nut

,label,quantity,unit
2 hams,,,
ENERC_KCAL,Energy,29354,kcal
PROCNT_KCAL,Calories from protein,11039,kcal
FAT_KCAL,Calories from fat,15505,kcal
CHOCDF_KCAL,Calories from carbohydrates,2810,kcal


In [10]:
# Desplegar DataFrame de nutrientes totales 
EDAMAM_consulta.df_Nutrition 

2 hams,label,quantity,unit
ENERC_KCAL,Energy,29354.4,kcal
FAT,Fat,1679.72,g
FASAT,Saturated,558.06,g
FAMS,Monounsaturated,798.603,g
FAPU,Polyunsaturated,205.481,g
CHOCDF,Carbs,684.936,g
FIBTG,Fiber,0,g
SUGAR,Sugars,0,g
PROCNT,Protein,2690.82,g
CHOLE,Cholesterol,9621.72,mg


In [11]:
# Desplegar DataFrame de nutrientes total diario
EDAMAM_consulta.df_totalDaily 

,label,quantity,unit
2 hams,,,
ENERC_KCAL,Energy,1467.72,%
FAT,Fat,2584.19,%
FASAT,Saturated,2790.3,%
CHOCDF,Carbs,228.312,%
FIBTG,Fiber,0,%
PROCNT,Protein,5381.64,%
CHOLE,Cholesterol,3207.24,%
NA,Sodium,7060.01,%
CA,Calcium,114.156,%


In [12]:
# Registro de Calorias
print('Calorías:',EDAMAM_consulta.Nutrient_Cal)

Calorías: 29354


## **Despliegue de los DataFrames de la API Food Database**

In [13]:
# Desplegar DataFrame de la base de datos de alimentos consultados
EDAMAM_consulta.df_food_table

,ENERGY (kcal),PROTEIN (g),FAT (g),CARBS (g),FIBER (g)
Pizza,268.00,10.36,12.28,29.02,2.2
"Coffe, Large",0.85,0.17,0.00,0.00,0
"Coffe, Medium",1.06,0.21,0.00,0.00,0
Sausage Pizza,280.00,11.50,12.35,30.62,2.3
Pepperoni Pizza,282.00,11.74,11.91,31.98,2.3
Stuffed Cheeze Pizza,274.00,12.23,11.63,30.00,1.7
Veggy Cheese Pizza,244.00,11.02,10.90,25.38,2.2
Nestle Coffee Mate Coffe Creamer Hazelnut,236.70,0.00,6.76,33.81,Unknown
Nestle Coffee Mate Coffe Creamer Hazelnut,236.70,0.00,6.76,33.81,Unknown
Thick Crust Sausage Pizza,282.00,11.06,12.94,30.36,2.3


## **Despliegue de los DataFrames de la API Recipe Search**

In [14]:
# Desplegar DataFrame para la búsqueda de recetas
EDAMAM_consulta.df_Recipe

ingredient name  \
Types                        Items                                          
Twinkie Tiramisu recipes     0      3 tablespoons instant espresso powder   
                             1                              3/4 cup sugar   
                             2              8 ounces whipped cream cheese   
                             3                 1 teaspoon vanilla extract   
                             4                     1 1/2 cups heavy cream   
...                                                                   ...   
Gramma Schuler&#39;s Pasties 9                               1/3 cup lard   
                             10                       1 tablespoon butter   
                             11                               1 cup flour   
                             12                         1/2 teaspoon salt   
                             13                    1/8 cup ice cold water   

                                    quantity     measure             food  \
Types                        Items                                          
Twinkie Tiramisu recipes     0      3.000000  tablespoon         espresso   
                             1      0.750000         cup            sugar   
                             2      8.000000       ounce     cream cheese   
                             3      1.000000    teaspoon  vanilla extract   
                             4      1.500000         cup      heavy cream   
...                                      ...         ...              ...   
Gramma Schuler&#39;s Pasties 9      0.333333         cup             lard   
                             10     1.000000  tablespoon           butter   
                             11     1.000000         cup            flour   
                             12     0.500000    teaspoon             salt   
                             13     0.125000         cup            water   

                                        weight          food category  \
Types                        Items                                      
Twinkie Tiramisu recipes     0       45.000000         coffee and tea   
                             1      150.000000                 sugars   
                             2      226.796185                 Cheese   
                             3        4.200000  Condiments and sauces   
                             4      357.000000                  Dairy   
...                                        ...                    ...   
Gramma Schuler&#39;s Pasties 9       68.333333                   Oils   
                             10      14.200000                  Dairy   
                             11     125.000000                 grains   
                             12       3.000000  Condiments and sauces   
                             13      29.625000                  water   

                                                                                image  
Types                        Items                                                     
Twinkie Tiramisu recipes     0      https://www.edamam.com/food-img/433/43301ebd4f...  
                             1      https://www.edamam.com/food-img/ecb/ecb3f5aaed...  
                             2      https://www.edamam.com/food-img/ddd/dddce306e7...  
                             3      https://www.edamam.com/food-img/90f/90f910b0bf...  
                             4      https://www.edamam.com/food-img/484/4848d71f6a...  
...                                                                               ...  
Gramma Schuler&#39;s Pasties 9      https://www.edamam.com/food-img/bc2/bc208be87a...  
                             10     https://www.edamam.com/food-img/713/71397239b6...  
                             11     https://www.edamam.com/food-img/b4c/b4c739e76a...  
                             12     https://www.edamam.com/food-img/694/6943ea5109...  
                             13     https://www.